This notebook will feature and explain work by [Chris Rackauckas](http://chrisrackauckas.com/) who has accepted a position in Applied Math starting January 2019 at MIT.

This notebook features discussion of [Stochastic pdes on gpus](http://www.stochasticlifestyle.com/solving-systems-stochastic-pdes-using-gpus-julia/).

We also point out the remarkable [ODE Chart](http://www.stochasticlifestyle.com/comparison-differential-equation-solver-suites-matlab-r-julia-python-c-fortran/) comparing all kinds of solvers.




## 1. First the @. macro

Steven Johnson discusses the uses and performance reasons for what is known as "broadcasting" in his [more dots blog](https://julialang.org/blog/2017/01/moredots) from January 2017.  More recently, one has the opportunity to use fewer dots!

In [53]:
o = ones(Int,5,5)

5×5 Array{Int64,2}:
 1  1  1  1  1
 1  1  1  1  1
 1  1  1  1  1
 1  1  1  1  1
 1  1  1  1  1

In [54]:
o*o

5×5 Array{Int64,2}:
 5  5  5  5  5
 5  5  5  5  5
 5  5  5  5  5
 5  5  5  5  5
 5  5  5  5  5

In [55]:
o.*o

5×5 Array{Int64,2}:
 1  1  1  1  1
 1  1  1  1  1
 1  1  1  1  1
 1  1  1  1  1
 1  1  1  1  1

In [56]:
@. o*o

5×5 Array{Int64,2}:
 1  1  1  1  1
 1  1  1  1  1
 1  1  1  1  1
 1  1  1  1  1
 1  1  1  1  1

In [57]:
exp(o)

5×5 Array{Float64,2}:
 30.4826  29.4826  29.4826  29.4826  29.4826
 29.4826  30.4826  29.4826  29.4826  29.4826
 29.4826  29.4826  30.4826  29.4826  29.4826
 29.4826  29.4826  29.4826  30.4826  29.4826
 29.4826  29.4826  29.4826  29.4826  30.4826

In [6]:
Σ = sum
Π = prod
Base.:*(x::Number,::typeof(!)) = factorial(x)
function (n::Int64)(::typeof(!))  # call overload
  factorial(n)
end
# could be 
#  (n::Int64)(::typeof(!)) = factorial(n)

In [7]:
x=3
x(!)

6

In [8]:
@which 3(!)

*(x::Number, ::typeof(!)) in Main at In[6]:3

In [9]:
?@.

```
@. expr
```

Convert every function call or operator in `expr` into a "dot call" (e.g. convert `f(x)` to `f.(x)`), and convert every assignment in `expr` to a "dot assignment" (e.g. convert `+=` to `.+=`).

If you want to *avoid* adding dots for selected function calls in `expr`, splice those function calls in with `$`.  For example, `@. sqrt(abs($sort(x)))` is equivalent to `sqrt.(abs.(sort(x)))` (no dot for `sort`).

(`@.` is equivalent to a call to `@__dot__`.)

# Examples

```jldoctest
julia> x = 1.0:3.0; y = similar(x);

julia> @. y = x + 3 * sin(x)
3-element Array{Float64,1}:
 3.5244129544236893
 4.727892280477045
 3.4233600241796016
```


In [76]:
factorial(5)

120

In [77]:
5!

LoadError: syntax: extra token "!" after end of expression

In [87]:
f(i) = i.(!)
f.([1:5;])

5-element Array{Int64,1}:
   1
   2
   6
  24
 120

In [89]:
(1:5) .* !

5-element Array{Int64,1}:
   1
   2
   6
  24
 120

In [10]:
Σ( o^k / k(!) for k ∈ 0:20 )

5×5 Array{Float64,2}:
 30.4826  29.4826  29.4826  29.4826  29.4826
 29.4826  30.4826  29.4826  29.4826  29.4826
 29.4826  29.4826  30.4826  29.4826  29.4826
 29.4826  29.4826  29.4826  30.4826  29.4826
 29.4826  29.4826  29.4826  29.4826  30.4826

In [11]:
ex = (:( (3)(!)))

:(3!)

In [12]:
function foo(ex::Expr)
  if ex.args[3]==:!
    println(factorial(ex.args[2]))
  end
end

foo (generic function with 1 method)

In [13]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol *
    2: Int64 3
    3: Symbol !


In [58]:
@. exp(o)

5×5 Array{Float64,2}:
 2.71828  2.71828  2.71828  2.71828  2.71828
 2.71828  2.71828  2.71828  2.71828  2.71828
 2.71828  2.71828  2.71828  2.71828  2.71828
 2.71828  2.71828  2.71828  2.71828  2.71828
 2.71828  2.71828  2.71828  2.71828  2.71828

In [59]:
1:5 .> 3

5-element BitArray{1}:
 false
 false
 false
  true
  true

In [61]:
float.(1:5 .>3)

5-element Array{Float64,1}:
 0.0
 0.0
 0.0
 1.0
 1.0

In [62]:
@. float(1:5>3)

5-element Array{Float64,1}:
 0.0
 0.0
 0.0
 1.0
 1.0

In [63]:
h(x) = x >0 ? x : 0 ## RelU

h (generic function with 1 method)

In [64]:
x = randn(10)

10-element Array{Float64,1}:
 -0.22909518903083093  
  1.0766400623127106   
  0.26933205331101656  
  2.2175659077791208   
  0.0042347489617710525
  0.6989836092219928   
 -0.8813035076174897   
  1.997209810988095    
 -1.0181457261507512   
 -0.18014238661082413  

In [65]:
@. h(x)

10-element Array{Real,1}:
 0                    
 1.0766400623127106   
 0.26933205331101656  
 2.2175659077791208   
 0.0042347489617710525
 0.6989836092219928   
 0                    
 1.997209810988095    
 0                    
 0                    

In [75]:
@. (1:5) + (1:5)'

5×5 Array{Int64,2}:
 2  3  4  5   6
 3  4  5  6   7
 4  5  6  7   8
 5  6  7  8   9
 6  7  8  9  10

## 2. Next the Array Partition

In [18]:
# using Pkg
# Pkg.add("RecursiveArrayTools")

In [19]:
using RecursiveArrayTools

In [9]:
 a = rand(2,2)
 b = randn(3)
 c = Diagonal(1:5)
[a,b,c][3][3,3]
(a,b,c)[1][1,2]

0.6560840548105469

In [97]:
a,b,c

([0.786116 0.546308; 0.0753156 0.933176], [-0.489238, -0.65838, -1.40055], [1 0 … 0 0; 0 2 … 0 0; … ; 0 0 … 4 0; 0 0 … 0 5])

In [98]:
u = ArrayPartition(a,b,c)

([0.786116 0.546308; 0.0753156 0.933176], [-0.489238, -0.65838, -1.40055], [1 0 … 0 0; 0 2 … 0 0; … ; 0 0 … 4 0; 0 0 … 0 5])

In [105]:
u[:]

32-element Array{Float64,1}:
  0.7861164537366732 
  0.07531556666765216
  0.5463081624668469 
  0.9331764440589279 
 -0.48923809396810464
 -0.658379501391394  
 -1.4005464083766674 
  1.0                
  0.0                
  0.0                
  0.0                
  0.0                
  0.0                
  ⋮                  
  0.0                
  0.0                
  0.0                
  0.0                
  0.0                
  4.0                
  0.0                
  0.0                
  0.0                
  0.0                
  0.0                
  5.0                

In [20]:
# u = ArrayPartition(A1,A2,A3,...)
# lets you get (A1,A2,...)[n][i,j]  by typing u[n,i,j]

In [21]:
a = [1 2 3;4 5 6]
b = [7 8 9;10 11 12]
u = ArrayPartition(a,b)

(9, 9)

In [22]:
const N = 100
A = zeros(N,N); B  = zeros(N,N); C = zeros(N,N);
u0 = ArrayPartition((A,B,C))

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

## A difficult to understand differential equation (but only because of the notation and the jargon, actually it's not so bad.)

$$A_t=DΔA+α_A(x)−β_AA−r_1AB+r_2 C$$
$$B_t=α_B−β_BB−r_1AB+r_2C $$
$$C_t=α_C−β_CC+r_1AB−r_2C$$

This is a PDE on a 2d grid evolving in time. 

$A,B,C$ are functions of x,y discretized as 2d arrays.
The time derivatives $A_t,B_t,C_t$ are also functions of x,y discretized as 2d arrays.

Δ is the ordinary Laplacian : $\frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2}.$ <br>
On a discretized grid of unit step size this is  sum of my NEWS neighbors (north,east,...) - 4*me

All the other variables are scalars, mostly independent of time and space except for the first $\alpha$ which will depend on the x coordinate (and can be stored as a vector, not a matrix, but not what he did in the code)

I kind of wish all scalar constants were small Latin or Greek letters but anyway ...

In [23]:
const D = 100  # D makes me think derivative, wish it weren't used
const α₂,α₃    = 1.0, 1.0      # Change in notation from α_B and  α_C probably because unavailable in unicode
const β₁,β₂,β₃ = 1.0, 1.0, 1.0 # Change in notation from β with subscripts A,B,C
const r₁,r₂ = 1.0, 1.0

(1.0, 1.0)

In [108]:
N = 100
#const X = reshape([i for i in 1:100 for j in 1:100],N,N)
#const Y = reshape([j for i in 1:100 for j in 1:100],N,N)
const X = [j for i∈1:N, j∈1:N]
const Y = X'
const α₁ = float.(X.≥80); # We will make it 1 if X≥80  (Not constant in space, but constant as an array)

In [110]:
α₁

100×100 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

## The Tridiagonal Approach to the Laplacian

In [25]:
using LinearAlgebra

In [111]:
Mx = Tridiagonal(ones(Int,N-1), fill(-2,N), ones(Int,N-1))
Mx[2,1] = 2
Mx[end-1,end] = 2
My = Mx'

100×100 Tridiagonal{Int64,Array{Int64,1}}:
 -2   2   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  …   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  1  -2   1   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅      ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   1  -2   1   ⋅   ⋅   ⋅   ⋅   ⋅      ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   1  -2   1   ⋅   ⋅   ⋅   ⋅      ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   1  -2   1   ⋅   ⋅   ⋅      ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅   1  -2   1   ⋅   ⋅  …   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   1  -2   1   ⋅      ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   1  -2   1      ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   1  -2      ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   1      ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  …   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅      ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅      ⋅   ⋅   ⋅   ⋅   

In [112]:
A = zeros(Int,N,N)
A[3,3]=1
A

100×100 Array{Int64,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  

In [113]:
# sum of my NEWS neighbors (north,east,...) - 4*me
Mx*A + A*My

100×100 Array{Int64,2}:
 0  0   0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0   1  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  -4  1  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0   1  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0   0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0   0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0   0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0   0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0   0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0   0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0   0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0   0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0   0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0 

In [114]:
# The reflecting conditions
A = zeros(Int,N,N)
A[1,1]=1
A[1,5]=1
A[1,100]=1
A
Mx*A + A*My

100×100 Array{Int64,2}:
 -4  2  0  1  -4  1  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  2  -4
  2  0  0  0   2  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0   2
  0  0  0  0   0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0   0
  0  0  0  0   0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0   0
  0  0  0  0   0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0   0
  0  0  0  0   0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0   0
  0  0  0  0   0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0   0
  0  0  0  0   0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0   0
  0  0  0  0   0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0   0
  0  0  0  0   0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0   0
  0  0  0  0   0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0   0
  0  0  0  0   0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0   0
  0  0  0  0   0  0  0  0  0  0  0  0   

Discussion --- storing all these numbers on the tridiagonal, using matmul???

## Evaluating the RHSs

In [115]:
function f(du,u,p,t)
  A,B,C = u.x
  dA,dB,dC = du.x
  DA = D*(M*A + A*M)
  ## The time changes
  @. dA = DA + α₁ - β₁*A - r₁*A*B + r₂*C 
  @. dB = α₂ - β₂*B - r₁*A*B + r₂*C
  @. dC = α₃ - β₃*C + r₁*A*B - r₂*C
end

f (generic function with 2 methods)

Some comments:
The M here should be Mx and My <br>
Possible the D* should be D.* but the @. would ruin the matmul (perhaps another reason matmul seems odd) <br>
Aha he remarks in the comments that non matmul is better <br>
Otherwise the steps are written in a "data parallel" manner

## Running the code in 1.0

In [31]:
using Pkg
Pkg.add("OrdinaryDiffEq")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [4]:
#######################################################
### Solve the PDE
#######################################################
 
using OrdinaryDiffEq, RecursiveArrayTools, LinearAlgebra
 
# Define the constants for the PDE
const α₂ = 1.0
const α₃ = 1.0
const β₁ = 1.0
const β₂ = 1.0
const β₃ = 1.0
const r₁ = 1.0
const r₂ = 1.0
 D = 100.0
const γ₁ = 0.1
const γ₂ = 0.1
const γ₃ = 0.1
const N = 100
#const X = reshape([i for i in 1:100 for j in 1:100],N,N)
#const Y = reshape([j for i in 1:100 for j in 1:100],N,N)
#const α₁ = 1.0.*(X.>=80)
const X = [i for i∈1:N, j∈1:N]
const Y = [j for i∈1:N, j∈1:N]
const α₁ = float.(X.≥80)
 
#const Mx = full(Tridiagonal([1.0 for i in 1:N-1],[-2.0 for i in 1:N],[1.0 for i in 1:N-1]))
const Mx = Tridiagonal([1.0 for i in 1:N-1],[-2.0 for i in 1:N],[1.0 for i in 1:N-1])
const My = copy(Mx)
Mx[2,1] = 2.0
Mx[end-1,end] = 2.0
My[1,2] = 2.0
My[end,end-1] = 2.0
 
# Define the initial condition as normal arrays
A = zeros(N,N); B  = zeros(N,N); C = zeros(N,N)
u0 = ArrayPartition((A,B,C))
 
const MyA = zeros(N,N);
const AMx = zeros(N,N);
const DA = zeros(N,N)
# Define the discretized PDE as an ODE function
function f(du,u,p,t)
  A,B,C = u.x
  dA,dB,dC = du.x
  mul!(MyA,My,A)  
  mul!(AMx,A,Mx)
  @. DA = D*(MyA + AMx)
  @. dA = DA + α₁ - β₁*A - r₁*A*B + r₂*C
  @. dB = α₂ - β₂*B - r₁*A*B + r₂*C
  @. dC = α₃ - β₃*C + r₁*A*B - r₂*C
end
 
# Solve the ODE
prob = ODEProblem(f,u0,(0.0,100.0))
@time sol = solve(prob,BS3(),progress=true,save_everystep=false,save_start=false)
 
using Plots; gr()
p1 = surface(X,Y,sol[end].x[1],title = "[A]")
p2 = surface(X,Y,sol[end].x[2],title = "[B]")
p3 = surface(X,Y,sol[end].x[3],title = "[C]")

 
#######################################################
### Solve the PDE using CLArrays
#######################################################
 
# using CLArrays
# gA = CLArray(A); gB  = CLArray(B); gC = CLArray(C)
# const gMx = CLArray(Mx)
# const gMy = CLArray(My)
# const gα₁ = CLArray(α₁)
# gu0 = ArrayPartition((gA,gB,gC))

# const gMyA = CLArray(MyA)
# const gAMx = CLArray(AMx)
# const gDA = CLArray(DA)
# function gf(du,u,p,t)
#   A,B,C = u.x
#   dA,dB,dC = du.x
#   mul!(gMyA,gMy,A)
#   mul!(gAMx,A,gMx)
#   @. gDA = D*(gMyA + gAMx)
#   @. dA = gDA + gα₁ - β₁*A - r₁*A*B + r₂*C
#   @. dB = α₂ - β₂*B - r₁*A*B + r₂*C
#   @. dC = α₃ - β₃*C + r₁*A*B - r₂*C
# end

# prob2 = ODEProblem(gf,gu0,(0.0,100.0))
# GPUArrays.allowslow(false)
# @time sol = solve(prob2,BS3(),progress=true,dt=0.003,adaptive=false,save_everystep=false,save_start=false)

# prob2 = ODEProblem(gf,gu0,(0.0,100.0))
# sol = solve(prob2,BS3(),progress=true,save_everystep=false,save_start=false)
# Adaptivity currently fails due to https://github.com/JuliaGPU/CLArrays.jl/issues/10
 
#######################################################
### Solve the SPDE
#######################################################
 


 11.674714 seconds (3.61 M allocations: 114.870 MiB, 0.22% gc time)


MethodError: MethodError: Cannot `convert` an object of type Surface{Array{Float64,2}} to an object of type Array{Float64,1}
Closest candidates are:
  convert(::Type{Array{T,N}}, !Matched::StaticArrays.SizedArray{S,T,N,M} where M) where {T, S, N} at /Users/alanedelman/.julia/packages/StaticArrays/Ze5H3/src/SizedArray.jl:62
  convert(::Type{Array{T<:Real,1}}, !Matched::AbstractRange{T<:Real}) where T<:Real at /Users/alanedelman/.julia/packages/Plots/7o1Vu/src/utils.jl:451
  convert(::Type{Array{T<:Real,1}}, !Matched::AbstractRange{S<:Real}) where {T<:Real, S<:Real} at /Users/alanedelman/.julia/packages/Plots/7o1Vu/src/utils.jl:452
  ...

In [8]:
using Pkg
Pkg.build("PyCall")
#using PyPlot
#using Pkg
#Pkg.add("PyPlots")

  Building Conda ─→ `~/.julia/packages/Conda/hsaaN/deps/build.log`
  Building PyCall → `~/.julia/packages/PyCall/rUul9/deps/build.log`


┌ Error: Error building `PyCall`: 
│ Solving environment: ...working... done
numpy-1.15.2         | 48 KB     | ########## | 100% 


mkl-2019.0           | 154.4 MB

mkl-2019.0           | 154.4 MB  | #####5     |  55% 
mkl_fft-1.0.6        | 137 KB    | ########## | 100% 
mkl_random-1.0.1     | 335 KB    | ########## | 100% 
blas-1.0             | 5 KB      | ########## | 100% 
libgfortran-3.0.1    | 495 KB    | ########## | 100% 
numpy-base-1.15.2    | 4.0 MB    | ########## | 100% 
intel-openmp-2019.0  | 1.0 MB    | ########## | 100% 
│ 
│ CondaHTTPError: HTTP 000 CONNECTION FAILED for url <https://repo.anaconda.com/pkgs/main/osx-64/mkl-2019.0-118.tar.bz2>
│ Elapsed: -
│ 
│ An HTTP error occurred when trying to retrieve this URL.
│ HTTP errors are often intermittent, and a simple retry will get you on your way.
│ 
│ 
│ 
│ 
│ ## Package Plan ##
│ 
│   environment location: /Users/alanedelman/.julia/packages/Conda/hsaaN/deps/usr
│ 
│   added / updated specs: 
│     - numpy
│ 
│ 
│ The following packages will be downloaded:
│ 
│     package                    |            build
│     ---------------------------|-----------------
│     numpy-1.15.2 

In [52]:
using Plots

┌ Info: Recompiling stale cache file /Users/alanedelman/.julia/compiled/v1.0/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1184


In [35]:
using StochasticDiffEq
 
function g(du,u,p,t)
  A,B,C = u.x
  dA,dB,dC = du.x
  @. dA = γ₁*A
  @. dB = γ₂*A
  @. dC = γ₃*A
end
 
prob3 = SDEProblem(f,g,u0,(0.0,100.0))
sol = solve(prob3,SRIW1())
#sol = solve(prob3,SRIW1(),progress=true,save_everystep=false,save_start=false)
 
# p1 = surface(X,Y,sol[end].x[1],title = "[A]")
# p2 = surface(X,Y,sol[end].x[2],title = "[B]")
# p3 = surface(X,Y,sol[end].x[3],title = "[C]")


ArgumentError: ArgumentError: Package StochasticDiffEq not found in current path:
- Run `Pkg.add("StochasticDiffEq")` to install the StochasticDiffEq package.


In [36]:
plot(p1,p2,p3)

UndefVarError: UndefVarError: plot not defined

In [37]:
Q = 1234

1234

In [38]:
foo(x) = x+Q

foo (generic function with 2 methods)

In [39]:
foo(100)

1334

In [40]:
Q = 0

0

In [41]:
foo(100)

100

In [42]:
dump(:((x)(!)))

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: Symbol x
    2: Symbol !


In [43]:
3(!)

6

In [44]:
x = 3
x(!)

6

In [45]:
Base.:*(x::Number,::typeof(!)) = factorial(x)

In [46]:
3(!)

6

In [47]:
4(!)

24

In [48]:
x=3
x*(!)

6

In [49]:
r"1" 

r"1"

In [50]:
@r_str "1"

r"1"

In [51]:
factorial"3!"

LoadError: UndefVarError: @factorial_str not defined